In [5]:
%%spark
from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
query1 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
query2 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
O9DataLake.register("sales_df",DataSource.LS, ResourceType.IBPL, query1) 
O9DataLake.register("DPBaseInputs",DataSource.LS, ResourceType.IBPL, query2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%spark
"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree

logger = logging.getLogger('o9_logger')

out_df = None
input_df = O9DataLake.get('sales_df')
predict_df = O9DataLake.get('DPBaseInputs')
if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)

else:
    out_df = None

O9DataLake.put('DPBaseOutput1',out_df)

O9DataLake.put('DPBaseOutput2',out_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…